# Stainlib: Usage example of normalizing with a target image

## Download Openslide
Openslide appears to be very difficult to get running on Window, however, it works well in linux. Good luck, have fun...
Try downloading the zipped binaries yourself and extract them in a local folder.
Then add the path to the dll directory, and to the PATH environmental variable for good measure. This worked for me, sometimes.

Install Openslide windows-binaries and add OPENSLIDE_PATH
* `https://openslide.org/download/#windows-binaries`
* `OPENSLIDE_PATH = r'the\openslide\path\bin'`

## MRXS files
mrxs files are sometimes not suitable for Openslide. If this happens, conversion from mrxs-mrxs by 3DHISTECH's Slide Converter solves the problem.
Probably have potential fixes in future:
https://github.com/openslide/openslide/issues/251

To get .mrxs:
* get api-key from Jan
* use python library from Ajey and Jonas (https://github.com/NKI-AI/slidescore-api)
* `pth_origin = r'where\the\image\path'`

In [1]:
import os  # Hope for error-free passage here...
OPENSLIDE_PATH = r'D:\Python\Yuan\openslide-win64\bin'
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [2]:
import sys
from pathlib import Path
pth = Path(Path.cwd().parent,'venv','src')  # Add stainlib package in the path
print(pth)
sys.path.append(str(pth))
import stainlib
from stainlib.utils.plot_utils import _plot_imagegrid
from stainlib.utils.stain_utils import is_uint8_image

D:\Python\Yuan\yuan_ki67\venv\src


In [3]:
import numpy as np
import matplotlib
matplotlib.use('QtAgg')
import matplotlib.pyplot as plt
import geojson
import configparser
import pickle
from pathlib import Path
# import tifffile
from tifffile import TiffWriter
from tifffile import imread
from tqdm import tqdm
%matplotlib notebook

In [18]:
slide_pth = Path( r'E:\1_DATA\BIF\yg_BIF\image\originalimages')
geojson_pth = Path( r'D:\QuPath\QuPath projects\BIF\yg.bif\Ki67_new\export')
tif_pth = Path( r'E:\1_DATA\BIF\yg_BIF\image\tile')  # remove later

slide_suffix = 'mrxs'
geojson_suffix = 'geojson'
tif_suffix = 'tif'  # remove later

def find_files(target_dir, target_suffix):
    find_res = []
    target_suffix_dot = "." + target_suffix
    walk_generator = os.walk(target_dir)
    for root_path, dirs, files in walk_generator:
        if len(files) < 1:
            continue
        for file in files:
            file_name, suffix_name = os.path.splitext(file)
            if suffix_name == target_suffix_dot:
                find_res.append(os.path.join(root_path, file))
    return find_res

# print(find_files(slide_pth, slide_suffix))
a = find_files(slide_pth, slide_suffix)
print(len(a))


25


In [4]:
## Open mrxs files
pth_origin = r'E:\1_DATA\BIF\yg_BIF\image\originalimages\16\T16-02242 A1 Ki 67.mrxs'
slide_origin = openslide.OpenSlide(pth_origin)
## Find the PhysicalSizeX and PhysicalSizeY
slide_props = slide_origin.properties 
X = slide_props['mirax.LAYER_0_LEVEL_0_SECTION.MICROMETER_PER_PIXEL_X']
Y = slide_props['mirax.LAYER_0_LEVEL_0_SECTION.MICROMETER_PER_PIXEL_Y']
x_offset = int(slide_props['mirax.NONHIERLAYER_0_LEVEL_0_SECTION.COMPRESSED_STITCHING_ORIG_SLIDE_SCANNED_AREA_IN_PIXELS__LEFT'])
y_offset = int(slide_props['mirax.NONHIERLAYER_0_LEVEL_0_SECTION.COMPRESSED_STITCHING_ORIG_SLIDE_SCANNED_AREA_IN_PIXELS__TOP'])
print(X,Y)
print(x_offset, y_offset)


## Open Tumor geojson files, they should be rectangle
## Find the correct annotation name
pth_annotation = r'D:\QuPath\QuPath projects\BIF\yg.bif\Ki67_new\export\T16-02242 A1 Ki 67.geojson'
annotation_name = 'Tumor Tissue'
annotation_tile = 'Sample Tile'


## Open sample region files (CHANGE later: Open Tumor sample region geojson files)
slide_tile = imread(r'E:\1_DATA\BIF\yg_BIF\image\tile\T16-02242 A1 Ki 67.tif')
print(slide_tile.shape)


## Load target model, the model is pre-trained in another notebook
pth_model = Path(r'D:\Python\Yuan\yuan_ki67\Notebooks\Models')
with open(Path(pth_model,"Macenko3.pickle"), "rb") as f:
    normalizer = pickle.load(f)


## Image output path
pth_out = r'E:\1_DATA\BIF\yg_BIF\image\1 target images'

0.242534722222222 0.242647058823529
30417 37332
(1024, 1024, 3)


In [5]:
## Find the position of annotation
# def annotation_coord(pth_annotation, annotation_name, annotation_tile, x_offset, y_offset):
#     with open(pth_annotation) as f:
#         tissue = geojson.load(f)
# #         print(tissue)
#         coordinates = file_stru(tissue, annotation_name, annotation_tile)
#         tissue_coordinates = coordinates[0]
#         tissue_region = get_coord(tissue_coordinates)
#         tissue_position = position_cal(tissue_region, x_offset, y_offset)
#         tile_coordinates = coordinates[1] 
#         tile_region = get_coord(tile_coordinates)
#         tile_position = position_cal(tile_region, x_offset, y_offset)
#         print(f'Tumor Tissue position (left, top, width, height): {tissue_position} \n'
#               f'Sample Tile position (left, top, width, height): {tile_position}')
#     return tissue_position, tile_position  # Don't forget to use later


def annotation_coord(pth_annotation, annotation_name, annotation_tile, x_offset, y_offset):
    with open(pth_annotation) as f:
        tissue = geojson.load(f)
#         print(tissue)
        coordinates = file_stru(tissue, annotation_name, annotation_tile)
        tissue_coordinates = coordinates[0]
        tissue_region = get_coord(tissue_coordinates)
        tissue_position = position_cal(tissue_region, x_offset, y_offset)
        print(f'Tumor Tissue position (left, top, width, height): {tissue_position} \n')
    return tissue_position



## 3 functions
def file_stru(tissue, annotation_name, annotation_tile):
    tissue_coords = []
    sample_coords = []    
    if tissue['type'] == 'FeatureCollection':
        pass
    else:
        raise ValueError('The annotataion type is not the FeatureCollection!')
    features_length = len(tissue['features'])
    print(f'There are {features_length} annotation(s) in this geojson file.')
    for feature in tissue['features']:
        if 'classification' in feature['properties']:
            if 'name' in feature['properties']['classification']:
                classification_name = feature['properties']['classification']['name']
                if classification_name == annotation_name:
                    tissue_coord = feature['geometry']['coordinates']
                    for coord in tissue_coord:
                        tissue_coords.append(coord)
                elif classification_name == annotation_tile:
                    sample_coord = feature['geometry']['coordinates']
                    for coord in sample_coord:
                        sample_coords.append(coord)
                else:
                    raise ValueError(f'The annotation name is wrong! Please check if it belongs to {annotation_name}.')
            else:
                raise ValueError('The annotation need a name!')
        else:
            raise ValueError('The annotations should have properties set. Please classified the annotation!')
    if not tissue_coords:
        raise ValueError(f'Warning! {annotation_name} annotation is empty.')
    if not sample_coords:
        print(f'Warning! {annotation_tile} annotation is empty.') # Remember to change raise ValueError
    return tissue_coords, sample_coords
        
def get_coord(coordinates):
    list_depth = len(np.array(coordinates).shape)
#     print(list_depth)
    new_coordinates = []
    for num in coordinates:
        for i in range(list_depth - 2):
            if i < (list_depth - 3):
                num = num[0]
            else:
                for j in num:
                    new_coordinates.append(j)
    return new_coordinates

def position_cal(point_list, x_offset, y_offset):
    x = []
    y = []
    for i,j in point_list:
        x.append(i)
        y.append(j)
    left = min(x) + x_offset - x_offset%256
    top = min(y) + y_offset - y_offset%256
    width = max(x) - min(x)
    height = max(y) - min(y)
    return left, top, width, height

In [6]:
## Find Tumor Tissue position
All_position = annotation_coord(pth_annotation, annotation_name, 
                                annotation_tile, x_offset, y_offset)

# annotation_position = All_position[0] # used later
# tile_position = All_position[1]

annotation_position = All_position
tif_left = annotation_position[0]
tif_top = annotation_position[1]
tif_width = annotation_position[2]
tif_height = annotation_position[3]
print(tif_left, tif_top, tif_width, tif_height)
# tile_left = tile_position[0]
# tile_top = tile_position[1]
# tile_width = tile_position[2]
# tile_height = tile_position[3]
# print(tile_left, tile_top, tile_width, tile_height)

There are 1 annotation(s) in this geojson file.
Warning! Sample Tile annotation is empty.
Tumor Tissue position (left, top, width, height): (66021, 113988, 26279, 36313) 

66021 113988 26279 36313


In [8]:
## Find Tumor Tissue region

def rgba2rgb(rgba, background=(255,255,255)):
    row, col, ch = rgba.shape
    if ch == 3:
        return rgba
    assert ch == 4, 'RGBA image has 4 channels.'
    rgb = np.zeros( (row, col, 3))
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]
    a = np.asarray(a) / 255.0
    R, G, B = background
    rgb[:,:,0] = r * a + (1.0 - a) * R
    rgb[:,:,1] = g * a + (1.0 - a) * G
    rgb[:,:,2] = b * a + (1.0 - a) * B
    return np.asarray(rgb, dtype='uint8')

slide_tif_whole = slide_origin.read_region((tif_left, tif_top), 0, (tif_width, tif_height))
slide_tif_rgba = np.asarray(slide_tif_whole)
slide_tif = rgba2rgb(slide_tif_rgba)

# slide_tile_whole = slide_origin.read_region((tile_left, tile_top), 0, (tile_width, tile_height))
# slide_tile = np.asarray(slide_tile_whole)
print(slide_tif.shape)
# plt.imshow(slide_tif)
# plt.show()

(36313, 26279, 3)


In [9]:
## Get source slide's stain vector and concentrations
## Standardize brightness (This step is optional but can improve the tissue mask calculation)
slide_tile_transform = stainlib.utils.stain_utils.LuminosityStandardizer.standardize(slide_tile)
slide_tile_matrix = stainlib.extraction.macenko_stain_extractor.MacenkoStainExtractor.get_stain_matrix(slide_tile_transform)
slide_tile_concentrations = stainlib.utils.stain_utils.get_concentrations(slide_tile_transform, slide_tile_matrix)
print(slide_tile_matrix)

[[0.61255646 0.77354654 0.1624818 ]
 [0.08075022 0.27214032 0.95886341]]


In [10]:
## Split slide to 100 tiles
size_x = int(slide_tif.shape[0]/1024)
size_y = int(slide_tif.shape[1]/1024)
if size_x > 0 and size_y > 0:
    spitsizes = [int(slide_tif.shape[0]/1024),int(slide_tif.shape[1]/1024)]
else:
    spitsizes = [1,1]
y = np.array_split(slide_tif, spitsizes[0], axis=0)
tiles = []
for x in y:
    tiles.append(np.array_split(x, spitsizes[1], axis=1))

In [11]:
## Normalization
transformed_tiles = []
for tile_row in tqdm(tiles, position=0):
    transformed_tile_row = []
    for tile in tqdm(tile_row, position=1, leave=False):
        to_transform = stainlib.utils.stain_utils.LuminosityStandardizer.standardize(tile)
        transformed = normalizer.transform(to_transform, slide_tile_matrix, slide_tile_concentrations)
        to_transform_final = np.transpose(transformed, (2,0,1))
        transformed_tile_row.append(to_transform_final)
    transformed_tiles.append(transformed_tile_row)

 12%|█████████████████████████▋                                                                                                                                                                                            | 3/25 [00:14<01:48,  4.94s/it]


 32%|████████████████████████████████████████████████████████████████████▍                                                                                                                                                 | 8/25 [00:40<01:24,  5.00s/it]


 52%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                      | 13/25 [01:27<02:10, 10.90s/it]


 72%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                           | 18/25 [02:30<01:37, 13.97s/it]


 92%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                 | 23/25 [03:59<00:36, 18.08s/it]


...

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                          | 20/25 [05:40<00:42,  8.44s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [05:44<00:00,  5.57s/it]


 12%|█████████████████████████▋                                                                                                                                                                                            | 3/25 [00:34<04:39, 12.71s/it]


 32%|████████████████████████████████████████████████████████████████████▍                                                                                                                                                 | 8/25 [02:02<05:03, 17.88s/it]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [3:20:06<00:00, 343.04s/it]


In [12]:
## Merge tiles to slide
rows=[]
for transformed_tile_row in transformed_tiles:
    rows.append(np.concatenate(transformed_tile_row, axis=2))
transformed_tile = np.concatenate(rows, axis=1)

In [14]:
## Save normalized slide
with TiffWriter(Path(pth_out,'a.ome.tif'), bigtiff=True) as tif:
    options = dict(photometric='rgb',tile=(1024, 1024), compression='jpeg',
                   metadata={'PhysicalSizeX': X, 'PhysicalSizeY': Y})
    tif.write(transformed_tile, subifds=4, **options)
    tif.write(transformed_tile[:, ::2, ::2], subfiletype=1, **options)
    tif.write(transformed_tile[:, ::4, ::4], subfiletype=1, **options)
    tif.write(transformed_tile[:, ::8, ::8], subfiletype=1, **options)
    tif.write(transformed_tile[:, ::16, ::16], subfiletype=1, **options)
print('Save Done')

Save Done
